In [3]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs
import zipfile
import pandas as pd

In [4]:
# URL of the page containing the links to the ZIP files
base_url = "https://www.omie.es"
page_url = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Day-ahead%20Market&parents%5B2%5D=3.%20Curves&dir=Monthly%20files%20with%20aggregate%20supply%20and%20demand%20curves%20of%20Day-ahead%20market%20including%20bid%20units&realdir=curva_pbc_uof"

In [5]:
# Folder where the ZIP files will be saved
download_folder = "data"
extracted_folder = os.path.join(download_folder, "extracted")

if not os.path.exists(download_folder):
    os.makedirs(download_folder)
if not os.path.exists(extracted_folder):
    os.makedirs(extracted_folder)

In [6]:
# Get page content
response = requests.get(page_url)
if response.status_code != 200:
    print(f"Error accessing the page: {response.status_code}")
    exit()

soup = BeautifulSoup(response.content, 'lxml')

In [7]:
# Find all links to ZIP files
zip_links = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href and "curva_pbc_uof_" in href and href.endswith(".zip"):
        full_url = urljoin(base_url, href)
        zip_links.append(full_url)

In [15]:
# Download each ZIP file found
for zip_url in zip_links:
    # Get file name from URL
    parsed_url = urlparse(zip_url)
    query_params = parse_qs(parsed_url.query)
    zip_name = query_params.get('filename', [None])[0]
    if zip_name:
        zip_path = os.path.join(download_folder, zip_name)
        print(f"Downloading {zip_path} from {zip_url}")

        # Download ZIP File
        response = requests.get(zip_url, stream=True)
        if response.status_code == 200:
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Download completed: {zip_path}")
        else:
            print(f"Error downloading {zip_url}: {response.status_code}")
    else:
        print(f"Could not define file name for URL: {zip_url}")

print("Download all files completed.")


Descargando data\curva_pbc_uof_202402.zip desde https://www.omie.es/es/file-download?parents%5B0%5D=curva_pbc_uof&filename=curva_pbc_uof_202402.zip
Descarga completa: data\curva_pbc_uof_202402.zip
Descargando data\curva_pbc_uof_202401.zip desde https://www.omie.es/es/file-download?parents%5B0%5D=curva_pbc_uof&filename=curva_pbc_uof_202401.zip
Descarga completa: data\curva_pbc_uof_202401.zip
Descargando data\curva_pbc_uof_202312.zip desde https://www.omie.es/es/file-download?parents%5B0%5D=curva_pbc_uof&filename=curva_pbc_uof_202312.zip
Descarga completa: data\curva_pbc_uof_202312.zip
Descargando data\curva_pbc_uof_202311.zip desde https://www.omie.es/es/file-download?parents%5B0%5D=curva_pbc_uof&filename=curva_pbc_uof_202311.zip


In [1]:
# Function to extract and process files
def extract_and_process_zip(zip_path):
    dataframes = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # List ZIP files
        for file_name in zip_ref.namelist():
            if file_name.endswith(".1"):
                # Extract file
                zip_ref.extract(file_name, extracted_folder)
                extracted_file_path = os.path.join(extracted_folder, file_name)
                # Read CSV file into a DataFrame
                df = pd.read_csv(extracted_file_path, sep=';')
                dataframes.append(df)
    return dataframes

In [8]:
# List all ZIP files in the downloads folder
zip_files = [f for f in os.listdir(download_folder) if f.endswith('.zip')]

In [10]:
# Process each ZIP file
all_dataframes = []
for zip_file in zip_files:
    zip_path = os.path.join(download_folder, zip_file)
    dataframes = extract_and_process_zip(zip_path)
    all_dataframes.extend(dataframes)


c:\Users\ajortiz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
# Print the number of extracted DataFrames
print(f"Total de archivos extraídos y cargados en DataFrames: {len(all_dataframes)}")

#You can access each DataFrame in the all_dataframes list
# For example, to display the first DataFrame:
if all_dataframes:
    print(all_dataframes[0].head())

# Guardar todos los DataFrames en un solo archivo CSV
#combined_df = pd.concat(all_dataframes, ignore_index=True)
#combined_df.to_csv(os.path.join(download_folder, "combined_data.csv"), index=False)
#print("Todos los DataFrames se han combinado y guardado en combined_data.csv")

Total de archivos extraídos y cargados en DataFrames: 85
  OMIE - Mercado de electricidad Fecha Emisi�n :30/11/2023 - 12:59 Unnamed: 2  \
0                           Hora                             Fecha       Pais   
1                              1                        01/12/2023         MI   
2                              1                        01/12/2023         MI   
3                              1                        01/12/2023         MI   
4                              1                        01/12/2023         MI   

  01/12/2023 Mercado diario            Unnamed: 5           Unnamed: 6  \
0     Unidad    Tipo Oferta  Energ�a Compra/Venta  Precio Compra/Venta   
1    TOTRM02              C                 137,6             1.500,00   
2    TOTRM01              C                 549,7             1.500,00   
3      BPC01              C                  16,9             1.002,00   
4     BPRM01              C                  25,3             1.002,00   

           